### Ref: https://keras-team.github.io/keras-tuner/examples/helloworld/

In [1]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=ce9347af28b0a09aeaa1e46ea010d43588cb4187afee7ea2657dacea07d355cc
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=284faaa6ddf592061eea7f5f802f480306824511437da75034d6d190d633c28f
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from kerastuner.tuners import RandomSearch

In [3]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
"""Basic case:
- We define a `build_model` function
- It returns a compiled model
- It uses hyperparameters defined on the fly
"""

def build_model(hp):
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 1e-5])),
                metrics=['accuracy'])
  return model

In [5]:
# Define tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='test_dir')

In [6]:
# Show stats
tuner.search_space_summary()

In [7]:
# Search for best params
tuner.search(x=x_train,
             y=y_train,
             epochs=3,
             validation_data=(x_test, y_test))

Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 1.1529 - accuracy: 0.6902 - val_loss: 0.4036 - val_accuracy: 0.8961
Epoch 2/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.4782 - accuracy: 0.8587 - val_loss: 0.2867 - val_accuracy: 0.9192
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.3806 - accuracy: 0.8878 - val_loss: 0.2384 - val_accuracy: 0.9310
Epoch 1/3
1875/1875 [==============================] - 18s 9ms/step - loss: 1.0887 - accuracy: 0.6849 - val_loss: 0.3979 - val_accuracy: 0.8950
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.4752 - accuracy: 0.8567 - val_loss: 0.2830 - val_accuracy: 0.9209
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.3795 - accuracy: 0.8853 - val_loss: 0.2344 - val_accuracy: 0.9319


Epoch 1/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7134 - accuracy: 0.7666 - val_loss: 0.2497 - val_accuracy: 0.9282
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.5527 - accuracy: 0.8243 - val_loss: 0.2058 - val_accuracy: 0.9394
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.4222 - accuracy: 0.8706 - val_loss: 0.1461 - val_accuracy: 0.9554
Epoch 1/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2877 - accuracy: 0.9127 - val_loss: 0.0890 - val_accuracy: 0.9744
Epoch 2/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.2075 - accuracy: 0.9408 - val_loss: 0.0732 - val_accuracy: 0.9774
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.2045 - accuracy: 0.9421 - val_loss: 0.0882 - val_accuracy: 0.9765


Epoch 1/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1806 - accuracy: 0.9460 - val_loss: 0.0433 - val_accuracy: 0.9853
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0801 - accuracy: 0.9762 - val_loss: 0.0353 - val_accuracy: 0.9881
Epoch 3/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0612 - accuracy: 0.9816 - val_loss: 0.0369 - val_accuracy: 0.9879
Epoch 1/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1933 - accuracy: 0.9403 - val_loss: 0.0460 - val_accuracy: 0.9853
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0816 - accuracy: 0.9766 - val_loss: 0.0406 - val_accuracy: 0.9867
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0605 - accuracy: 0.9817 - val_loss: 0.0291 - val_accuracy: 0.9900


Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3915 - accuracy: 0.8840 - val_loss: 0.0972 - val_accuracy: 0.9711
Epoch 2/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.1367 - accuracy: 0.9601 - val_loss: 0.0572 - val_accuracy: 0.9826
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0986 - accuracy: 0.9710 - val_loss: 0.0444 - val_accuracy: 0.9849
Epoch 1/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.3786 - accuracy: 0.8882 - val_loss: 0.1021 - val_accuracy: 0.9693
Epoch 2/3
1875/1875 [==============================] - 18s 9ms/step - loss: 0.1366 - accuracy: 0.9596 - val_loss: 0.0605 - val_accuracy: 0.9809
Epoch 3/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1005 - accuracy: 0.9704 - val_loss: 0.0458 - val_accuracy: 0.9866


INFO:tensorflow:Oracle triggered exit


In [8]:
tuner.results_summary()

In [9]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps.get('learning_rate')

0.001

In [12]:
# Get best model
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [14]:
# Evaluate 
score = best_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.029104068875312805
Test accuracy: 0.9900000095367432


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
# model = tuner.hypermodel.build(best_hps)

# model.fit(x=x_train,
#              y=y_train,
#              epochs=3,
#              validation_data=(x_test, y_test))